Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

*   **Año:** [2024]
*   **Alumno/a:** [Mateo Moreno]
*   **Legajo:** [46583878]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [ ]:
from pydantic import BaseModel

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [ ]:
class Client(BaseModel):
  dni: int
  nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [ ]:
user = Client(dni = 39755010, nacionalidad = "Argentina")

print(user)

dni=39755010 nacionalidad='Argentina'


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [ ]:
user = Client(dni = "39755010")
user = Client(nacionalidad = "Argentina")

print(user)

ValidationError: 1 validation error for Client
nacionalidad
  Field required [type=missing, input_value={'dni': '39755010'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [ ]:
from pydantic import BaseModel, validator
from datetime import date

class Client(BaseModel):
    dni: int
    nacionalidad: str
    fecha_registro: date

    @validator("dni", pre=True)
    def dni_positivo(cls, dni):
      if dni < 0:
        raise ValueError("El DNI debe ser un número positivo")
      return dni

    @validator("nacionalidad", pre=True)
    def nacionalidad_valida(cls, pais):
      nacionalidades_validas = ["Argentina", "Brasil", "Chile", "Uruguay", "Colombia", "Peru",  "España", "Mexico", "Estados Unidos", "Canada"]
      if pais not in nacionalidades_validas:
        raise ValueError("Nacionalidad inválida.")
      return pais

    @validator("fecha_registro", pre=True)
    def fecha_valida(cls, fecha):
      if fecha > date.today():
        raise ValueError("La fecha de registro no puede ser en el futuro.")
      return fecha

<ipython-input-9-501bfbc88d34>:9: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator("dni", pre=True)
<ipython-input-9-501bfbc88d34>:15: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator("nacionalidad", pre=True)
<ipython-input-9-501bfbc88d34>:22: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration

Pasamos a los casos donde funciona y donde no.


In [ ]:
try:
    cliente_valido = Client(dni = 39755010, nacionalidad = "Argentina", fecha_registro = date(2023, 1, 1))
    print("Cliente válido:", cliente_valido)  # Imprime solo si no hay error
except ValueError as exc:
    print("Error:", exc)

Cliente válido: dni=39755010 nacionalidad='Argentina' fecha_registro=datetime.date(2023, 1, 1)


In [ ]:
try:
    cliente_invalido = Client(dni = -123, nacionalidad = "Brasil", fecha_registro = date(2022, 5, 10))
except ValueError as exc:
    print("Error:", exc)  # Solo imprime el error si hay uno

Error: 1 validation error for Client
dni
  Value error, El DNI debe ser un número positivo [type=value_error, input_value=-123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [ ]:
try:
    cliente_invalido = Client(dni = 45678901, nacionalidad = "Japon", fecha_registro = date(2021, 12, 31))
except ValueError as exc:
    print("Error:", exc)  # Solo imprime el error si hay uno

Error: 1 validation error for Client
nacionalidad
  Value error, Nacionalidad inválida. [type=value_error, input_value='Japon', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [ ]:
!pip install pandera

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandera as pa
import pandas as pd
from datetime import date

nacionalidades_validas = ["Argentina", "Brasil", "Chile", "Uruguay", "Colombia", "Peru", "España", "Mexico", "Estados Unidos", "Canada"]

ClientSchema = pa.DataFrameSchema({
    "dni": pa.Column(int, pa.Check(lambda s: (s > 0).all())),  # Use .all() for Series
    "nacionalidad": pa.Column(str, pa.Check(lambda s: s.isin(valid_nationalities).all())),  # Use .isin() and .all()
    "fecha_registro": pa.Column(date, pa.Check(lambda s: (s <= date.today()).all()))  # Use .all() for Series
})

Pasamos a los casos que funciona y los que no.

In [ ]:
data = {"dni": [39755010], "nacionalidad": ["Argentina"], "fecha_registro": [pd.to_datetime("2023-01-01").date()]} # Convert Timestamp to date
df = pd.DataFrame(data)
try:
    validated_df = ClientSchema.validate(df)
    print("DataFrame validado:\n", validated_df)
except pa.errors.SchemaError as exc:
    print("Error de validación:", exc)

DataFrame validado:
         dni nacionalidad fecha_registro
0  39755010    Argentina     2023-01-01


In [ ]:
data = {"dni": [-123], "nacionalidad": ["Brasil"], "fecha_registro": [pd.to_datetime("2022-05-10").date()]}
df = pd.DataFrame(data)
try:
    validated_df = ClientSchema.validate(df)
    print("DataFrame validado:\n", validated_df)
except pa.errors.SchemaError as exc:
    print("Error de validación:", exc)

Error de validación: Column 'dni' failed series or dataframe validator 0: <Check <lambda>>


In [ ]:
data = {"dni": [39755010], "nacionalidad": ["Japon"], "fecha_registro": [pd.to_datetime("2022-05-10").date()]}
df = pd.DataFrame(data)
try:
    validated_df = ClientSchema.validate(df)
    print("DataFrame validado:\n", validated_df)
except pa.errors.SchemaError as exc:
    print("Error de validación:", exc)

Error de validación: Column 'nacionalidad' failed series or dataframe validator 0: <Check <lambda>>
